# OVERVIEW

## Setting up Libraries, S3 client, and Dask Dashboard

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import dask_geopandas as dg
import utils as ut

### ATTENTION:
User must follow below instructions and replace the AWS access key and secret access key. This is crucial for the analysis to work.

##### Utilizing data from: https://source.coop/repositories/wherobots/usa-structures/description

All data on Source Cooperative, are hosted on AWS S3 bucket. In order to access them, you need credentials that you can generate on Source Cooperative website. Atfer logging in, click on your name at the top right corner, and then click on your username. Then navigate to "Manage" page on the left side. At the bottom of this page you will find a section called "API Keys". If no key has been generated before, generate a new one and then copy the values for each of the following keys, and paste them in the following cell.

source.coop website: https://source.coop/

###### Source: https://github.com/github.com/HamedAlemo/vector-data-tutorial/scalable_vector_analysis.ipynb

In [ ]:
##################################
#   Read Above 'ATTENTION' Note  #
##################################

AWS_ACCESS_KEY_ID = "<YOUR ACCESS KEY>"
AWS_SECRET_ACCESS_KEY = "<YOUR SECRET ACCESS KEY>"

In [ ]:
import boto3
s3_client = boto3.client('s3',
                         aws_access_key_id = AWS_ACCESS_KEY_ID, 
                         aws_secret_access_key = AWS_SECRET_ACCESS_KEY,
                         endpoint_url='https://data.source.coop'
                        )

In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
print(client.dashboard_link)

##### Local path for downloading the data

If running the analysis file within the 'saved'//mounted folder and do NOT wish to save the raw data to your computer - delete the
first local_path and uncomment (remove initial #) on the second one. NOTE: you will have to re-install each time you load this image up if you do that.

In [ ]:
local_path = "./data/"    # saves data to machine
# local_path = '/home/gisuser/data/'   # deletes data after closing the container

## US Structures Data (Installing all of it)

Change the blocksize value if your computer is stronger. This currently sets each block to 16 Megabytes. More details can be found here (ctrl + f 'blocksize'): https://coderzcolumn.com/tutorials/python/dask-dataframes-guide-to-work-with-large-tabular-datasets

Install time will vary. As of 12/9/2024, the dataset is 10 geoparquets each equating to ~2 Gigabytes. This took ~40 minutes to install on my personal machine: ~ 3 Yr Old Laptop, Windows 11, 16 Gb RAM (Docker is limited to 12 Gb), Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz, 2592 Mhz, 6 Core(s), 12 Logical Processor(s)

In [ ]:
structure_ddf = ut.get_US_structures_all(s3_client, local_path, blocksize = "16M") # "256M" is regular block size

In [ ]:
structure_ddf.head(3)

In [ ]:
structure_ddf.tail(3)

Notice some columns aren't showing, lets see what they are:

In [ ]:
view_middle_columns = structure_ddf.columns[:20]
structure_ddf[view_middle_columns].head(5)

We can also further investigate the impacts of blocksize:

In [ ]:
partition40 = structure_ddf.partitions[40].compute()
partition40.head(5)

partition40.shape

Each partition has 16mb of data, roughly 873,670 rows. This will vary based on the blocksize chosen when calling the ut.get_US_structures_all() function.

## Census Data

In [ ]:
### Download

In [ ]:
# THIS CAN BE NEXT-STEPS // FURTHER FUTURE ANALYSIS. It does not need to be completed 100%
Review Census data within Worcester. Grab the shapefiles for two of the census blocks. spatial filter & intersection with the dataset again.

Then review and draw relationships from the data

##### Also, create a function to download country wide census data (depending on the size. Maybe just do cities/Massachusetts and just provide a link)

In [1]:
### Understanding it (comparing GeoId)

## Data Checks (for spatial joins)

### If you go on to review another state. You may need to re-project the CRS for the entire dataset. The following is a good site for visually seeing what the EPSG covers
https://spatialreference.org/ref/epsg/2249/

##### Coordinate Reference System (CRS)
Since this dataset captures "every structure larger than 450 square feet" we have a base minimum, however, to get specific building sizes, or plot, we will need to make sure the CRS is for the specific region/area we are analyzing. We will re-project based on the 'geometries' column.

First, we will check to see if we need to project to a CRS for the first time, or re-project the current one. To check if the data has a CRS, the following code will suffice.

In [ ]:
structure_ddf.crs

We will have to re-project for an appropraite EPSG for our area of interest. The entire dataset is in: 4326. 

Since we are analyzing Massachusetts (City: Worcester, Town: Norwood), and the shapefile we retreived from the City of Worcester was in 2249, we will use 2249.

EPSG:2249 covers both Massachusett counties we analyze (Worcester and Norfolk), this is in US Survey feet which means we don't need to worry about converting from meters to feet. 

In [ ]:
structure_ddf.to_crs(epsg=2249)
# NOTE: if you analyze another area that this crs is not good for, you will want to make adjustments. For simplicity, we change the crs for the 
# entire dataset.

Calculate the square footage for each building and make this stored in a new column 'structure_size':

In [ ]:
structure_ddf['structure_size'] = structure_ddf.geometry.area

In [2]:
### geometry column

## Dataset Integrity

This section shows an example of questions you need to be asking each time you load in a dataset. It is important to verify work so you know the best way to analyze the dataset


Our dataset has columns that specify the state and/or city that a structure is in. However, these columns have blanks as not every structure is identified. For this reason, I want to investigate how many buildings don't get identified.

I will use the city of worcester shapefile from the city government's website. Whenever downloading a shapefile, make sure you get all the other files that come with it:
.shp, .cpg, .dbf, .prj, .shx, and .xml

I'll compare the shapefile to 2 different dataframes:

1. The first dataframe will make a dataframe of buildings/rows that are identified as Massachusetts. Then from that dataframe, I will grab a new one for buildings/rows that are identified as Worcester.

2. The second dataframe will be the entire dataset, no filtering based on columns.

Then I will compute the number of rows for each spatial join with the Worcester Shapefile. Then a user can decide if one is 'better' than the other. You also have to consider computational power. If you only lose 100 out of 100,000 buildings, is it worth it to save computational power? I think yes, but it all depends on your analysis and abilities. 

You will not see the outputs from most '.compute()' lines of code, that is due to a weak computational power on my personal machine. Any computation should be ran with caution, especially those on the entire dataset. I will try to include warnings before each compute to remind the user.

In [5]:
# Compare 2 below with above 1. Take best of both!

In [ ]:
OVERVIEW:
- This is testing the usefulness of the datasets columns for specifying if a structure is in a specific state/city.



### If you go on to review another state. You may need to re-project the CRS for the entire dataset. The following is a good site for visually seeing what the EPSG covers
https://spatialreference.org/ref/epsg/2249/

### Checking the validity of the 'PROP_CITY' column

##### Compare the number of rows when using 'PROP_CITY' == 'Worcester' VS a shapefile of Worcester as a Spatial Filter and finding intersections
(Using massachusetts_ddf ensures not grabbing Worcester or Boston city data for other states)

We will use .shape to compare the number of rows

To ensure we use the correct boundaries for Worcester, we will download a shapefile of the layout. This will be downloaded to the 'data' folder and not done in an automatic coding format. However, if you find a different shapefile/city, the same approach can be used. Just mount a folder when you run the docker container from a directory that has the associated city boundary shape files, then you can change the 'dg_read_file' filepath. (Future Implementation: Use selenium to download this file within docker so that users don't need to have data downloaded with the Docker Image)

https://opendata.worcesterma.gov/datasets/worcesterma::city-boundary-1/about

In [ ]:
massachusetts_ddf = structure_ddf[structure_ddf['PROP_ST'] == 'Massachusetts']
worcester_PROP_CITY_ddf = massachusetts_ddf[massachusetts_ddf['PROP_CITY'] == 'Worcester']

In [ ]:
worcester_boundary = dg.read_file(f"./data/Worcester/City_Boundary.shp", chunksize = 75000) # chunksize specifies number of rows per chunk 

##### If the above line of code gave you an error similar to:

ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/envs/us_structures_analysis/share/proj failed

Try re-running it

or

Copy the folder that contains all of the shapefile files outside the mounted folder, so it is in the main directory.

For instance. I use a mounted 'saved' folder, with a 'data' folder inside that contains all the geoparquets plus a 'Worcester' folder which contains all the files for the Worcester shapefile. I copied the 'Worcester' folder so that it is outside of the 'saved' folder.

In [ ]:
# confirm crs
worcester_boundary.crs

In [ ]:
worcester_boundary.compute() # insight to the shapefile layout.

In [ ]:
#AS NOTED IN THE DATA CHECKS PORTION, we must check the geometry files are proper for the join.
# the structure_ddf.head(5) line and the above worcester_boundary.compute() will show how the geometry columns are named. The prints confirm they are
# the same.

structure_ddf = structure_ddf.set_geometry('geometry')
worcester_boundary = worcester_boundary.set_geometry('geometry')
print(structure_ddf.geometry.name)
print(worcester_boundary.geometry.name)

##### Computing Disclaimer
Here we will count the number of rows that each spatial join gives. The .compute() statements will provide the answers. Run at your own risk (if dask workers 'die' you will likely have to shutdown the docker container and open a new one)

In [ ]:
worcester_entire_ddf = dg.sjoin(structure_ddf, worcester_boundary, how='inner', predicate='within') # within makes sure the whole structure_ddf 
                                                                                                       # row is inside the worcester_boundary

# Understand this approach may mean some of the structure_ddf rows near the boundary won't be included if the shapefile is off by just a little.

worcester_PROP_ddf = dg.sjoin(worcester_PROP_CITY_ddf, worcester_boundary, how='inner', predicate='within')

In [ ]:
entire_df_worcester_rows = worcester_entire_ddf.count().compute()
entire_df_worcester_rows

In [ ]:
filtered_by_columns_worcester_rows = worcester_PROP_ddf.count().compute()
filtered_by_columns_worcester_rows

## Town Example

The goal of adding a town example is that it will have less computation requirements than a city. I also show some potential plots that can be done. Note: my computer still could not handle this! Also, this is relying on filtering dataset by column names, but the zip codes seemed more filled in (PROP_ZIP) than state or city.

User could also grab a shapefile for Norwood and use the original structure_ddf to spatial join with it.

In [ ]:
norwood_ddf = massachusetts_ddf[massachusetts_ddf['PROP_ZIP'] == '02062']

##### Find number of structures in Norwood that are part of dataset (Computing Disclaimer)

In [ ]:
norwood_ddf.shape[0].compute()

##### Find the average structure size (Computing Disclaimer)

In [ ]:
norwood_ddf['structure_size'].mean().compute()

##### Make a histogram of structure sizes (Computing Disclaimer)

In [ ]:
norwood_structure_sorted = norwood_ddf.sort_values(by='structure_size', ascending=False).compute()

# binLength = len(bldg_norwood_sorted) // 25
plt.hist(norwood_structure_sorted['structure_size'], bins = 300, edgecolor='black')
plt.xlabel('Area (in ft^2)')
plt.ylabel('Frequency')
plt.title('area_in_square_feet for Norwood Buildings')
plt.show()

## Future Implementation

- Utilize selenium python package and create a function that can go onto Worcester website and download the shapefile
- Provide plotting examples (was hesitant to attempt since I can't compute anything)